In [1]:
from aesara.tensor.random.op import RandomVariable
from pymc.aesaraf import floatX, intX
from aesara.tensor.var import TensorVariable
from typing import Tuple

import numpy as np
import aesara.tensor as at
import pymc as pm
from pymc.distributions import distribution, multivariate
from pymc.distributions.continuous import Flat, Normal, HalfNormal, Uniform, get_tau_sigma

In [2]:
class AR1rv(RandomVariable):
    name = "AR1"
    ndim_supp = 0
    ndims_params = [0, 0, 0, 0, 0]
    dtype = "floatX"
    _print_name = ("AR1", "\\operatorname{AR1}")
    
    
    def __call__(self, phi, init=0.0, mu=0.0, sigma=1.0,*args,**kwargs) -> TensorVariable:
        return super().__call__(phi, init, mu, sigma,*args, **kwargs)

    
    @classmethod
    def rng_fn(
            cls,
            rng: np.random.default_rng(),
            phi: float,
            init: float,
            mu: np.ndarray,
            sigma: np.ndarray,
            size: Tuple[int, ...],
    ) -> np.ndarray:

        phi = np.asarray(phi)
        
        if init!=0.0:
            y = np.asarray([init])
        else:
            y = rng.normal(0.0, 1.0,size=1)
        
        y[-1]
        for i in range(1, size[0]):
            y = np.append(y, mu + phi*y[-1] + rng.normal(0.0, sigma))
            
        return y

In [3]:
class AR1(distribution.Continuous):
    rv_op = AR1rv()
    r"""
    Autoregressive with order 1.

    Parameters
    ----------
    phi: tensor
        Autoregressive coefficient.
    sigma: float
        Standard deviation of noise (sigma > 0, default: 1.0).
    mu: float
        Mean or constant (default: 0.0)
    size: int
        Number of observations in time series.
        
    Examples
    --------
    
    .. code-block:: python
    
        #Generate an AR1 process
        Y = AR1.dist(phi=.50, size=500)
        
        #Estimate AR1 parameter
        with pm.Model() as ar1:
        
            #priors
            phi = Uniform("phi",-0.99,0.99,shape=1)
            
            #sampling and results
            ar1 = AR1("AR1", phi=phi, observed = Y)
            trace = pm.sample(1000, tune=500, target_accept=0.99,return_inferencedata=True)
            az.summary(trace)
            
    """
    @classmethod
    def dist(cls, phi,*args,**kwargs):
        return super().dist([phi],*args,**kwargs)

    def logp(obs, phi, mu=0.0, sigma=1.0, *args,**kwargs):
        
        n = obs.shape[0]
        epsilon = at.ivector()
        
        for i in range(1, n):
            epsilon = at.sub(Y, mu + phi*obs[-1])
        
        logp = at.sum(pm.logp(Normal.dist(mu=nu, sigma=sigma,size=n), epsilon))

        return logp


In [4]:
Y = AR1.dist(phi=[.50], size=500, init=0.3, mu=0.0, sigma=1.0).eval()

In [5]:
import statsmodels.api as sm

mod = sm.tsa.statespace.SARIMAX(Y, order=(1, 0, 0))

res_mle = mod.fit(disp=False)
print(res_mle.summary())

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  500
Model:               SARIMAX(1, 0, 0)   Log Likelihood                -694.258
Date:                Wed, 26 Jan 2022   AIC                           1392.516
Time:                        01:18:13   BIC                           1400.945
Sample:                             0   HQIC                          1395.823
                                - 500                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.5082      0.041     12.388      0.000       0.428       0.589
sigma2         0.9404      0.061     15.390      0.000       0.821       1.060
Ljung-Box (L1) (Q):                   0.02   Jarque-

In [6]:
with pm.Model() as AR:
    phi = Uniform("phi",-0.99,0.99,shape=1)
    AR1 = AR1("AR1", phi=phi, observed = Y)
    trace = pm.sample(1000, tune=500, target_accept=0.99,return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


TypeError: Cannot convert Type TensorType(float64, vector) (of Variable TensorConstant{[ 3.000000..07616e-01]}) into Type TensorType(float64, (True,)). You can try to manually convert TensorConstant{[ 3.000000..07616e-01]} into a TensorType(float64, (True,)).